In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from python_utils import *
from tqdm.notebook import tqdm, trange

### Case 1: $F = \mathbb Q(E[12])$

In [2]:
e0 = 12
def get_ramification_dataset1(l):
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':divisors(12*l)}
    ep_range[2] = {'good':[e for e in divisors(pow(2,8) * pow(3,2)) if e%2 == 0], 'multi':[e for e in divisors(24*l) if e%2 == 0]}
    ep_range[3] = {'good':[e for e in divisors(pow(2,6) * pow(3,2)) if e%2 == 0], 'multi':[e for e in divisors(48*l) if e%2 == 0]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[(l-1)*e for e in divisors(12*l)]}
    return ep_range

# compute log-volume
my_log_volumes1 = dict()
for p in tqdm(primes(300)):
    if p >= 5 and p <= 30:
        my_log_volumes1[p] = compute_log_volume(get_ramification_dataset1, e0, p)
        print(p, my_log_volumes1[p])
    elif p > 30:
        my_log_volumes1[p] = compute_log_volume_fast(get_ramification_dataset1, e0, p)

  0%|          | 0/62 [00:00<?, ?it/s]

5 133.92056029892038
7 116.89711856627233
11 126.66197956597263
13 135.53407835194733
17 157.00550521728516
19 167.61314166656254
23 190.32667244864382
29 225.4548740511201


In [3]:
# upper bounds for $\log(x^{3r} y^s)$ when $\max\{3r, s\} \ge u0_apos \ge u_0$,
# slso for $\log(x^{3r})$ when $3r \ge u0_apos$; and for $\log(y^s)$ when $s \ge u0_apos$.
def get_impossible_interval_1(log_volumes,S,k,u0,u0_apos):
    S = sorted(S)
    n = len(S)
    if n < k:
        return []
    S1 = [3]
    e0 = 12
    p_N = 2
    n0 = 27

    # define p_*, u_*, n_*
    p0 = S[0]
    u0 = u0
    n1_S = p0
    nk_S = prod(S[:k])
    # N'
    u0_apos = u0_apos
    n1_S_apos = max(u0_apos, p0)
    # define a_*, b_*
    a1 = 0 
    a2 = 0
    a3 = 0
    a4 = []
    for l in S:
        a1 += (l*l+5*l)/(l*l+l-12)*(1-1/e0/l)
        a2 += log_volumes[l] + log(2) # by the definition of Vol(l)
        a3 += log(l)
        a4.append((l+5)*(l-1)/(l*l+l-12)/e0)
    a1 = a1 / n * 6
    a2 = a2 / n * 6
    a3 = a3
    a4 = min(a4) * 6
    a5 = sum([log(p) for p in S1])
    #  define b_1, b'_1, b_2, impossible inteval
    b1 = max(k/n + (a1-a4) / n1_S, a1/u0)
    b1_apos = max(k/n + (a1-a4) / n1_S_apos, a1/u0_apos)
    b2 = a1/u0 * log(n0) + a2 + a1*a3 + (a1-a4)*a5
    if b1 > 1 or b1_apos >= 1:
        return []
    upper_bound = nk_S * log(p_N)
    lower_bound = b2 / (1-b1_apos)
    if lower_bound > upper_bound:
        return []
    return [lower_bound,upper_bound]


def get_upper_bound_1(log_volumes,u0,u0_apos):
    my_primes = [p for p in primes(150) if p >= 11 and p != 13]
    # choice of possible set S
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    # impossible inteval for log(N')
    for S in arr_S:
        for k in [2,3,4,5]:
            interval = get_impossible_interval_1(log_volumes,S,k,u0,u0_apos)
            if len(interval)==2:
                intervals.append(interval)
    res = merge_intervals(intervals)
    if len(res) == 0:
        return -999
    return res[-1][0] + log(27)


when $r \in \{4,5\}$, $7\le s\le 3677$

In [13]:
# when r in [4, 5], we have t >= 7
# compute upper bounds for log(y^s), s >= u0_apos = 3600
u0_apos = 3600
bound = get_upper_bound_1(my_log_volumes1,min(3*4, u0_apos),u0_apos)
print(u0_apos,f"{bound + 1e-3: .3f}")
# when s >= 3600, we have log(2^s) <= log(y^s) < 2549 (since y >= 2 by Catalan's conjecture), hence $s < 2550 / log(2) < 3678, 7 <= s <= 3677
print(2549 / log(2))

3600  2548.496
3677.429659225968


when $r, s \ge 7$, we have $7 \le r,s \le 1226$

In [46]:
# s >= 7, compute upper bounds for log(x^{3r}), 3r >= u0_apos = 2000
u0_apos = 3600
bound = get_upper_bound_1(my_log_volumes1,7,u0_apos)
print(u0_apos,f"{bound + 1e-3: .3f}")
# when r >= 3600, we have log(2^{3r}) <= log(x^{3r}) < 2551, hence $r < 2551 / 3 / log(2) < 1227, 7 <= r <= 1226, similarly, we have 7 <= s <= 1226
print(2551 / 3 / log(2))

3600  2550.535
1226.771683102582


In [79]:
# r,s >= 7 => log(x^{3r} y^s) < 31000
get_upper_bound_1(my_log_volumes1,7,7)

30676.40156919736

In [81]:
# r,s >= 7 => log(x^{3r}) < 4439 => log(x^r) < 1480
bound = get_upper_bound_1(my_log_volumes1,7,21)
bound, bound / 3


(4438.961498373133, 1479.6538327910441)

### Case 2: $F=\mathbb Q(E[4])$

In [96]:
e0 = 4
def get_ramification_dataset2(l):
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':divisors(4*l)}
    ep_range[2] = {'good':[e for e in divisors(96) if e%2 == 0], 'multi':[e for e in divisors(8*l) if e%2 == 0]}
    ep_range[3] = {'good':[e for e in divisors(12) if e%2 == 0], 'multi':[2*e for e in divisors(8*l)]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[(l-1)*e for e in divisors(4*l)]}
    return ep_range

# compute log-volume
my_log_volumes2 = dict()
for p in tqdm(primes(300)):
    if p >= 5 and p <= 30:
        my_log_volumes2[p] = compute_log_volume(get_ramification_dataset2, e0, p)
        print(p, my_log_volumes2[p])
    elif p > 30:
        my_log_volumes2[p] = compute_log_volume_fast(get_ramification_dataset2, e0, p)

  0%|          | 0/62 [00:00<?, ?it/s]

5 59.17843935222741
7 50.356443373450205
11 51.87927802801745
13 54.9948057061493
17 61.945242964890056
19 65.7568310535161
23 73.48046840034378
29 85.47170755570102


suppose that $r\ge 7$, $s \ge 21$, then $r,s \le 1226$. 

Let $x_0, y_0$ be the coprime to $2$ part of $x,y$.

In [97]:
# upper bounds for $\log(x1^{3r} y1^s)$ when $\max\{3r, s\} = s_0 = u_0$,
# slso for $\log(x^{3r1})$ when $u0_apos = 3r \ge s_0$; and for $\log(y1^s)$ when $s \ge s_0$.
def get_impossible_interval_2(log_volumes,S,k,u0,u0_apos):
    S = sorted(S)
    n = len(S)
    if n < k:
        return []
    S1 = [3]
    e0 = 2
    p_N = 3
    n0 = 27

    # define p_*, u_*, n_*
    p0 = S[0]
    u0 = u0
    n1_S = p0
    nk_S = prod(S[:k])
    # N'
    u0_apos = u0_apos
    n1_S_apos = u0_apos
    # define a_*, b_*
    a1 = 0 
    a2 = 0
    a3 = 0
    a4 = []
    for l in S:
        a1 += (l*l+5*l)/(l*l+l-12)*(1-1/e0/l)
        a2 += log_volumes[l] + log(2) # by the definition of Vol(l)
        a3 += log(l)
        a4.append((l+5)*(l-1)/(l*l+l-12)/e0)
    a1 = a1 / n * 6
    a2 = a2 / n * 6
    a3 = a3
    a4 = min(a4) * 6
    a5 = sum([log(p) for p in S1])
    #  define b_1, b'_1, b_2, impossible inteval
    b1 = max(k/n + (a1-a4) / n1_S, a1/u0)
    b1_apos = max(k/n + (a1-a4) / n1_S_apos, a1/u0_apos)
    b2 = a1/u0 * log(n0) + a2 + a1*a3 + (a1-a4)*a5
    if b1 > 1 or b1_apos >= 1:
        return []
    upper_bound = nk_S * log(p_N)
    lower_bound = b2 / (1-b1_apos)
    if lower_bound > upper_bound:
        return []
    return [lower_bound,upper_bound]

def get_upper_bound_2(log_volumes,u0,u0_apos):
    # choose l \nmid r * s
    my_primes = [p for p in primes(150) if p >= 11 and p != 13]
    # choice of possible set S
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    # impossible inteval for log(N')
    for S in arr_S:
        for k in [2,3,4,5]:
            interval = get_impossible_interval_2(log_volumes,S,k,u0,u0_apos)
            if len(interval)==2:
                intervals.append(interval)
    res = merge_intervals(intervals)
    if len(res) == 0:
        return -999
    return res[-1][0] + log(27)

since $\min\{3r, s\} \ge 21$, we have $\log(x_0^{3r} y_0^s) \le 1481$, $\log(x_0^r) \le 494$; $\log(y_0^{3s} x_0^r) \le 1208$, $\log(y_0^s) \le 403$. 

In [107]:
# x^{3r} y^s
bound = get_upper_bound_2(my_log_volumes2,21,21)
print(bound, bound / 3)
# y^{3s} x^r
bound = get_upper_bound_2(my_log_volumes2,7,63)
print(bound, bound / 3)




1480.5316202428494 493.5105400809498
1207.9398418843236 402.64661396144123
